# Reduce Marketing Waste

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Data Cleaning 

In [6]:
#Load the dataset

train = pd.read_csv(r"D:\DS\HackerEarth_DS\Reducing_marketing_waste_challenge\dataset\train.csv")
test = pd.read_csv(r"D:\DS\HackerEarth_DS\Reducing_marketing_waste_challenge\dataset\test.csv")

train = train[(train["Success_probability"]>0)&(train["Success_probability"]<=100)]

test['Internal_rating'] = test['Internal_rating'].astype(int)
test['Internal_rating'] = test['Internal_rating'].replace(-1,1)
test['Internal_rating'] = test['Internal_rating'].replace(82,np.nan)
test['Internal_rating'] = test['Internal_rating'].fillna(method='ffill').astype(int)

train.shape
#test.head(20)

(6422, 23)

In [3]:
#Look for discrepancies in the data and rectify them

train['Resource'] = train['Resource'].str.replace('We have all the requirements','Yes')
train['Resource'] = train['Resource'].str.replace('Deliverable','Yes')
train['Resource'] = train['Resource'].str.replace('Not enough','No')
train['Resource'] = train['Resource'].str.replace('Cannot deliver','No')

test['Resource'] = test['Resource'].str.replace('We have all the requirements','Yes')
test['Resource'] = test['Resource'].str.replace('Deliverable','Yes')
test['Resource'] = test['Resource'].str.replace('Not enough','No')
test['Resource'] = test['Resource'].str.replace('Cannot deliver','No')

train['Designation'] = train['Designation'].replace({'Chairman/CEO/President':'Chairman/CEO/President',
                                              'CEO/Chairman/President':'Chairman/CEO/President',
                                              'Chief Executive Officer':'CEO',
                                              'Vice President / GM (04-present) : VP Sales and Marketing (01-04)':'Vice President/GM'})

test['Designation'] = test['Designation'].replace({'Chairman/CEO/President':'Chairman/CEO/President',
                                              'CEO/Chairman/President':'Chairman/CEO/President',
                                              'Chief Executive Officer':'CEO',
                                              'Vice President / GM (04-present) : VP Sales and Marketing (01-04)':'Vice President/GM'})

In [4]:
train.columns

Index(['Deal_title', 'Lead_name', 'Industry', 'Deal_value', 'Weighted_amount',
       'Date_of_creation', 'Pitch', 'Contact_no', 'Lead_revenue',
       'Fund_category', 'Geography', 'Location', 'POC_name', 'Designation',
       'Lead_POC_email', 'Hiring_candidate_role', 'Lead_source',
       'Level_of_meeting', 'Last_lead_update', 'Internal_POC', 'Resource',
       'Internal_rating', 'Success_probability'],
      dtype='object')

In [5]:
train.isnull().sum()
#test.isnull().sum()

Deal_title                 0
Lead_name                  0
Industry                   1
Deal_value                46
Weighted_amount          474
Date_of_creation           0
Pitch                      0
Contact_no                 0
Lead_revenue               0
Fund_category              0
Geography                878
Location                   9
POC_name                   8
Designation                0
Lead_POC_email             0
Hiring_candidate_role      0
Lead_source                0
Level_of_meeting           0
Last_lead_update         583
Internal_POC               0
Resource                 138
Internal_rating            0
Success_probability        0
dtype: int64

In [6]:
#filling nans for numerical data types 

train['Deal_value'] = train['Deal_value'].str.replace(r"\$","")
train['Weighted_amount'] = train['Weighted_amount'].str.replace(r"\$","")
train_weighted_mean = train['Weighted_amount'].astype(float).mean()
train['Weighted_amount'] = train['Weighted_amount'].astype(float).fillna(train_weighted_mean)
train_deal_mean = train['Deal_value'].astype(float).mean()
train['Deal_value'] = train['Deal_value'].astype(float).fillna(train_deal_mean)


# filling nans for categorical data types : Industry , POC_name , Location

train['Industry'] = train['Industry'].fillna(value='Unknown')
train['POC_name'] = train['POC_name'].fillna(value='Unknown')
train['Location'] = train['Location'].fillna(value='Unknown')

#also datetime is modified to yield only "the year of the deal"

train['Date_of_creation'] = train['Date_of_creation'].str.split("-")
train['Date_of_creation'] = train['Date_of_creation'].str[0]
train['Date_of_creation'] = pd.to_datetime(train['Date_of_creation'])
train['Date_of_creation'] = train['Date_of_creation'].dt.year

#filling nans for below columns using backward/forward fill
train['Geography'] = train['Geography'].fillna(method='ffill')
train['Last_lead_update'] = train['Last_lead_update'].fillna(method='ffill')
train['Resource'] = train['Resource'].fillna(method='bfill')

#look if any nan values are left to be dismissed
train.isnull().sum()

Deal_title               0
Lead_name                0
Industry                 0
Deal_value               0
Weighted_amount          0
Date_of_creation         0
Pitch                    0
Contact_no               0
Lead_revenue             0
Fund_category            0
Geography                0
Location                 0
POC_name                 0
Designation              0
Lead_POC_email           0
Hiring_candidate_role    0
Lead_source              0
Level_of_meeting         0
Last_lead_update         0
Internal_POC             0
Resource                 0
Internal_rating          0
Success_probability      0
dtype: int64

In [7]:
train.Resource.value_counts()

No     3237
Yes    3185
Name: Resource, dtype: int64

In [8]:
#repeating the process for test data as well
#filling nans for numerical data types 

test['Deal_value'] = test['Deal_value'].str.replace(r"\$","")
test['Weighted_amount'] = test['Weighted_amount'].str.replace(r"\$","")
test_weighted_mean = test['Weighted_amount'].astype(float).mean()
test['Weighted_amount'] = test['Weighted_amount'].astype(float).fillna(train_weighted_mean)
test_deal_mean = test['Deal_value'].astype(float).mean()
test['Deal_value'] = test['Deal_value'].astype(float).fillna(train_deal_mean)

# filling nans for categorical data types : Industry , POC_name , Location

test['Industry'] = test['Industry'].fillna(value='Unknown')
test['POC_name'] = test['POC_name'].fillna(value='Unknown')
test['Location'] = test['Location'].fillna(value='Unknown')

#also datetime is modified to yield only "the year of the deal"

test['Date_of_creation'] = test['Date_of_creation'].str.split("-")
test['Date_of_creation'] = test['Date_of_creation'].str[0]
test['Date_of_creation'] = pd.to_datetime(test['Date_of_creation'])
test['Date_of_creation'] = test['Date_of_creation'].dt.year

#filling nans for below columns using backward/forward fill
test['Geography'] = test['Geography'].fillna(method='ffill')
test['Last_lead_update'] = test['Last_lead_update'].fillna(method='ffill')
test['Resource'] = test['Resource'].fillna(method='bfill')

#check for nan values
test.isnull().sum()

Deal_title               0
Lead_name                0
Industry                 0
Deal_value               0
Weighted_amount          0
Date_of_creation         0
Pitch                    0
Contact_no               0
Lead_revenue             0
Fund_category            0
Geography                0
Location                 0
POC_name                 0
Designation              0
Lead_POC_email           0
Hiring_candidate_role    0
Lead_source              0
Level_of_meeting         0
Last_lead_update         0
Internal_POC             0
Resource                 0
Internal_rating          0
dtype: int64

In [9]:
#Encoding the categorical columns 

object_cols = [ col for col in train.columns if train[col].dtype == 'object' ]
good_label_cols = [ col for col in object_cols if set(train[col])== set(test[col]) ]
bad_label_cols = list(set(object_cols)-set(good_label_cols))

label_train = train.drop(bad_label_cols,axis=1)
label_test = test.drop(bad_label_cols,axis=1)

label_encoder = LabelEncoder()
label_encoder_2 = LabelEncoder()
label_encoder_3 = LabelEncoder()
label_encoder_4 = LabelEncoder()
label_encoder_5 = LabelEncoder()
label_encoder_6 = LabelEncoder()
label_encoder_7 = LabelEncoder()
label_encoder_8 = LabelEncoder()
label_encoder_9 = LabelEncoder()
label_encoder_10 = LabelEncoder()
label_encoder_11 = LabelEncoder()
label_encoder_12 = LabelEncoder()

In [10]:
label_train['Pitch'] = label_encoder.fit_transform(label_train['Pitch'])
label_train['Lead_revenue'] = label_encoder_2.fit_transform(label_train['Lead_revenue'])
label_train['Fund_category'] = label_encoder_3.fit_transform(label_train['Fund_category'])
label_train['Geography'] = label_encoder_4.fit_transform(label_train['Geography'])
label_train['Designation'] = label_encoder_5.fit_transform(label_train['Designation'])
label_train['Lead_source'] = label_encoder_6.fit_transform(label_train['Lead_source'])
label_train['Level_of_meeting'] = label_encoder_7.fit_transform(label_train['Level_of_meeting'])
label_train['Last_lead_update'] = label_encoder_8.fit_transform(label_train['Last_lead_update'])
label_train['Internal_POC'] = label_encoder_9.fit_transform(label_train['Internal_POC'])
label_train['Resource'] = label_encoder_10.fit_transform(label_train['Resource'])
label_train['Internal_rating'] = label_encoder_11.fit_transform(label_train['Internal_rating'])
label_train['Date_of_creation'] = label_encoder_12.fit_transform(label_train['Date_of_creation'])

label_test['Pitch'] = label_encoder.transform(label_test['Pitch'])
label_test['Lead_revenue'] = label_encoder_2.transform(label_test['Lead_revenue'])
label_test['Fund_category'] = label_encoder_3.transform(label_test['Fund_category'])
label_test['Geography'] = label_encoder_4.transform(label_test['Geography'])
label_test['Designation'] = label_encoder_5.transform(label_test['Designation'])
label_test['Lead_source'] = label_encoder_6.transform(label_test['Lead_source'])
label_test['Level_of_meeting'] = label_encoder_7.transform(label_test['Level_of_meeting'])
label_test['Last_lead_update'] = label_encoder_8.transform(label_test['Last_lead_update'])
label_test['Internal_POC'] = label_encoder_9.transform(label_test['Internal_POC'])
label_test['Resource'] = label_encoder_10.transform(label_test['Resource'])
label_test['Internal_rating'] = label_encoder_11.transform(label_test['Internal_rating'])
label_test['Date_of_creation'] = label_encoder_12.transform(label_test['Date_of_creation'])

In [11]:
good_label_cols

['Pitch',
 'Lead_revenue',
 'Fund_category',
 'Geography',
 'Designation',
 'Lead_source',
 'Level_of_meeting',
 'Last_lead_update',
 'Internal_POC',
 'Resource']

In [12]:
label_test.columns

Index(['Deal_value', 'Weighted_amount', 'Date_of_creation', 'Pitch',
       'Lead_revenue', 'Fund_category', 'Geography', 'Designation',
       'Lead_source', 'Level_of_meeting', 'Last_lead_update', 'Internal_POC',
       'Resource', 'Internal_rating'],
      dtype='object')

In [13]:
label_train

,Deal_value,Weighted_amount,Date_of_creation,Pitch,Lead_revenue,Fund_category,Geography,Designation,Lead_source,Level_of_meeting,Last_lead_update,Internal_POC,Resource,Internal_rating,Success_probability
0,320506.0,2.067264e+06,1,1,1,1,1,5,3,2,7,12,0,2,73.60
1,39488.0,2.408768e+05,0,1,2,3,0,3,2,0,3,5,0,4,58.90
2,359392.0,2.407926e+06,0,0,2,3,1,6,1,0,2,18,0,3,68.80
3,76774.0,4.683214e+05,2,1,2,2,1,1,0,1,3,5,1,0,64.50
4,483896.0,1.573616e+06,0,1,1,2,0,5,3,1,9,53,0,3,62.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7000,152908.0,9.709658e+05,1,0,0,0,1,2,3,1,1,20,0,0,62.70
7001,479541.0,2.685430e+06,1,0,0,2,0,0,2,1,2,42,1,1,57.40
7003,220208.0,1.453373e+06,1,1,0,0,0,0,1,1,2,5,1,2,26.35
7004,253608.0,1.573616e+06,1,0,0,1,1,6,1,2,2,35,1,0,70.60


In [14]:
#fitting the model

X = label_train
y = label_train.Success_probability
X.drop(['Success_probability'], axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X,y,train_size=0.8, random_state=0)
model = RandomForestRegressor(n_estimators=50,criterion='mae',random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_val)
print(mean_absolute_error(y_val, preds))

7.377578210116736


In [15]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(model, random_state=0).fit(X_val, y_val)
eli5.show_weights(perm, feature_names = X_val.columns.tolist())

/Users/sanketsmac/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight,Feature
0.1076 ± 0.0470,Geography
0.0317 ± 0.0574,Fund_category
0.0262 ± 0.0490,Internal_rating
0.0219 ± 0.0239,Lead_source
0.0185 ± 0.0150,Deal_value
0.0163 ± 0.0161,Lead_revenue
0.0159 ± 0.0507,Level_of_meeting
0.0111 ± 0.0211,Weighted_amount
0.0074 ± 0.0094,Designation
0.0050 ± 0.0137,Internal_POC


In [16]:
#Derive the predictions

preds_test = model.predict(label_test)
preds_test

array([56.421, 64.248, 73.5  , ..., 45.561, 58.322, 56.91 ])

In [17]:
preds_test.shape

(2093,)

In [18]:
#creating the submission file

output = pd.DataFrame({'Deal_title': test.Deal_title,
                       'Success_probability': preds_test.round(2)})
output.to_csv('submission.csv', index=False)

In [19]:
submission = pd.read_csv('/Users/sanketsmac/Downloads/HackerEarth_DS/Reducing_marketing_waste_challenge/dataset/sample_submission.csv')
submission.head()

,Deal_title,Success_probability
0,TitleM5DZY,48.6
1,TitleKIW18,33.9
2,TitleFXSDN,43.8
3,TitlePSK4Y,39.5
4,Title904GV,37.4


In [20]:
#cross checking

answer = pd.read_csv('/Users/sanketsmac/Downloads/HackerEarth_DS/Reducing_marketing_waste_challenge/submission.csv')
answer

,Deal_title,Success_probability
0,TitleAD16O,56.42
1,TitleOW6CR,64.25
2,TitleVVJQ5,73.50
3,TitleUS8NA,65.21
4,Title5VGWW,62.16
...,...,...
2088,Title2R8VU,62.58
2089,Title7HCNJ,52.93
2090,TitleCD5YZ,45.56
2091,Title8OKXL,58.32
